In [3]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import time

In [4]:
start = time.time()
# 인기순 정렬
targetSite = 'https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=all&genreCode=&orderTypeCode=sale&is&isFinished=false'
driver = webdriver.Chrome()
driver.get(targetSite)
wait = WebDriverWait(driver, 10)

#//*[@id="content"]/p/a[1] # 2
#//*[@id="content"]/p/a[2] # 3
#//*[@id="content"]/p/a[3] # 4
#//*[@id="content"]/p/a[4] # 5
#//*[@id="content"]/p/a[5] # 6
#//*[@id="content"]/p/a[6] # 7
#//*[@id="content"]/p/a[6] # 8
#//*[@id="content"]/p/a[6] # 9


# 페이지 수에 따라 설정
total_pages = 5000 // 25 # 400 페이지 10.000개

Titles = []  # 제목 저장할 리스트
Genre = []  # 장르 저장할 리스트
Author = []  # 작가 저장할 리스트
Start = []  # 연재 시작일
Last = []  # 마지막 연재일
Count = []  # 연재수
Good = []  # 추천수
Point = []  # 평점
Comment = []  # 댓글 수
n=0
for page in range(1, total_pages + 1):
    #print(page)
    
    # 25개의 목록을 순차적으로 클릭
    for i in range(1, 26):
        # 각 목록의 XPath
        link_xpath = f'//*[@id="content"]/div/ul/li[{i}]/div/h3/a' # 페이지별 1~25번 링크
        
        # 크롤링 작업 수행
        try:
            # 목록 클릭
            element = driver.find_element(By.XPATH, link_xpath)
            element.click()
            
            if "login" in driver.current_url or "adult" in driver.current_url: # 19금, 로그인 필요한 경우
                # 이후 전처리 할 떄 쉽게 하도록 19금 통일
                Titles.append("19금")
                Genre.append("19금")
                Author.append("19금")
                Point.append("19금")
                Start.append("19금")
                Last.append("19금")
                Count.append("19금")
                Good.append("19금")
                Comment.append("19금")
                print(f'{n+1}번, 19금입니다.')
                n += 1
                driver.back()
                continue      
            
            if "stopSale" in driver.current_url: # 오래되서 판매 중지됨
                Titles.append("판매중지")
                Genre.append("판매중지")
                Author.append("판매중지")
                Point.append("판매중지")
                Start.append("판매중지")
                Last.append("판매중지")
                Count.append("판매중지")
                Good.append("판매중지")
                Comment.append("판매중지")
                print(f'{n+1}번, 판매 중지 되었습니다.')
                n += 1
                driver.back()
                continue
            
            # 연재 시작일과 마지막 연재일 수집
            # 정렬 기준이 랜덤이라 Start, Last에 들어가는게 반대로 들어가는 경우가 있음 (최신순, 1화부터)
            a = driver.find_element(By.XPATH, '//*[@id="content"]/table/thead/tr/th[1]/div')  # 최신순
            b = driver.find_element(By.XPATH, '//*[@id="content"]/table/thead/tr/th[2]/div')  # 1화부터
            
            if a.get_attribute('class') == 'on _changeTicketSortOrder(DESC)':  # 최신순 정렬
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Last.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                b.click()
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Start.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                driver.back()
            
            elif b.get_attribute('class') == 'on _changeTicketSortOrder(ASC)':  # 1화부터 정렬
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Start.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                a.click()
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Last.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                driver.back()
            
            try:
                Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text)
                Good.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[2]/ul/li[2]/div/a/em').text)
            except:
                Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text)
                Good.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[2]/ul/li[2]/div/a/em').text)
            
            try:
                Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text)
            except:
                Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text)
            
            Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text)
            Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text)
            Count.append(driver.find_element(By.XPATH, '//*[@id="content"]/h5/strong').text)
            
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="cbox_module"]/div/div[1]/span')))
            Comment.append(driver.find_element(By.XPATH, '//*[@id="cbox_module"]/div/div[1]/span').text) 
            
            
            print(f'{n+1}번, page:{page}. 제목: {Titles[n]}, 장르: {Genre[n]}, 작가: {Author[n]}, 시작일: {Start[n]}, 최신화: {Last[n]}, 연재수: {Count[n]}, 추천수: {Good[n]}, 평점: {Point[n]}, 댓글: {Comment[n]}')
            n += 1
            driver.back()
            
            # 페이지 로드 대기 (필요에 따라 조정)
            time.sleep(1)
        
        except Exception as e:
            print(f"Error at page {page}, item {i}: {e}")
            break
    # for 1~25번 반복 ======================================================
    
    # 다음 페이지로 이동
    next_link_xpath = f'//*[@id="content"]/p/a[{6 if page >= 6 else page}]'
    
    try:
        next_page_element = driver.find_element(By.XPATH, next_link_xpath)
        next_page_element.click()
        
        # 페이지 로드 대기 (필요에 따라 조정)
        time.sleep(0.5)
    
    except Exception as e:
        #print(f"Error navigating to next page {page}: {e}")
        break
        
# 드라이버 종료
#driver.quit()

end = time.time()


1번, page:1. 제목: 구르미 그린 달빛 [독점], 장르: 로맨스, 작가: 윤이수, 시작일: (2013.10.04.), 최신화: (2014.10.10.), 연재수: 131, 추천수: 14,999, 평점: 9.3, 댓글: 6,519
2번, page:1. 제목: 허니허니 웨딩 [독점], 장르: 로맨스, 작가: 노승아, 시작일: (2015.09.01.), 최신화: (2016.02.16.), 연재수: 98, 추천수: 9,231, 평점: 9.5, 댓글: 8,869
3번, page:1. 제목: 장씨세가 호위무사 [독점], 장르: 무협, 작가: 조형근, 시작일: (2014.11.05.), 최신화: (2023.06.29.), 연재수: 360, 추천수: 6,877, 평점: 9.3, 댓글: 18,912
4번, page:1. 제목: 수라왕 [삽화본], 장르: 무협, 작가: 이대성, 시작일: (2013.01.15.), 최신화: (2015.02.02.), 연재수: 309, 추천수: 3,291, 평점: 8.5, 댓글: 5,258
5번, page:1. 제목: 원하는 건 너 하나 [독점], 장르: 로맨스, 작가: 달콤J, 시작일: (2015.02.03.), 최신화: (2015.09.26.), 연재수: 96, 추천수: 6,418, 평점: 9.3, 댓글: 3,254
6번, page:1. 제목: 낙향문사전 [독점], 장르: 무협, 작가: 최현우(언라이팅), 시작일: (2013.09.03.), 최신화: (2018.03.08.), 연재수: 494, 추천수: 5,494, 평점: 9.5, 댓글: 18,071
7번, page:1. 제목: 해시의 신루 [독점], 장르: 로맨스, 작가: 윤이수, 시작일: (2015.04.02.), 최신화: (2016.03.28.), 연재수: 150, 추천수: 6,492, 평점: 9.5, 댓글: 5,549
8번, page:1. 제목: 패왕연가 [독점], 장르: 무협, 작가: 장영훈, 시작일: (2014.04.03.), 최신화: (2015.09.03.), 연재수: 300

In [5]:
df = pd.DataFrame({"제목": Titles, "장르": Genre, "작가": Author, "연재 시작일": Start, "마지막 연재일": Last, "회수": Count, "댓글수": Comment, "추천수": Good, "평점": Point})
df.index = df.index + 1

In [6]:
df.to_csv('./data/NSeries_10,000.txt')